# Action Rules

Research Questions:
1. How to change Cited from 0 to 1?

## Load data
The CORD-19 dataset is primarily composed of scientific articles related to coronaviruses that span from the 1970's to 2020, the beginning of the COVID-19 outbreak. The target variable in this dataset represents either a low or high citation rate.

In [1]:
import pandas as pd
import numpy as np
import json
pd.options.display.max_columns = None

data = pd.read_csv("c19_llm_features.csv")
data

,title,abstract,cited,rigor,novelty,grammar,replicability,accessibility,mathematics,computer_sciences,physical_sciences,chemical_sciences,earth_and_related_sciences,biological_sciences,other_natural_sciences,civil_engineering,electrical_engineering,mechanical_engineering,chemical_engineering,materials_engineering,environmental_engineering,evironmental_biotechnology,industrial_biotechnology,nano-technology,other_engineering_and_technologies,basic_medicine,clinical_medicine,health_sciences,medical_biotechnology,other_medical_sciences,agriculture_forestry_and_fisheries,animal_and_dairy_science,veterinary_science,agricultural_biotechnology,other_agricultural_sciences,psychology,economics_and_business,education,sociology,law,political_science,social_and_economic_geography,media_and_communications,other_social_sciences,history_and_archaeology,languages_and_literature,philosophy_ethics_and_religion,arts,other_humanities_and_the_arts,applied_research,conclusive_research,correlational_research,descriptive_research,ethnographic_research,experiment,exploratory_research,focus_groups,interviews,mixed_methods,observation,qualitative_research,quantitative_research,questionnaires,secondary_research,survey_methodology
0,The 2019 Sklare Address: How Gender and Family...,Family has resurfaced in many ways as its cont...,0,medium,medium,1,0,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ethical issues in the access to emergency care...,Patients who are undocumented immigrants (UIs)...,0,medium,low,0,0,high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Social Media Engagement and Influenza Vaccinat...,BACKGROUND: Vaccines are one of the most impor...,0,medium,low,0,0,high,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,1,0,0,1,0,1,1,0,0,1
3,Application of Open-Source Software in Knowled...,"Knowledge graph (KG), as a new type of knowled...",0,low,low,0,0,high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SARS-CoV-2 infection in nonhuman primates alte...,The current pandemic of coronavirus disease (C...,0,high,high,0,1,high,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,1,0,0,1,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,A fast and robust iterative algorithm for pred...,BACKGROUND: Improving accuracy and efficiency ...,1,high,high,0,1,high,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0
2994,Correcting misperceptions of exponential coron...,The most effective way to stem the spread of a...,1,high,high,0,1,high,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2995,Action Mechanisms of Lithium Chloride on Cell ...,Transmissible gastroenteritis virus (TGEV) is ...,1,high,high,0,1,high,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Drop unused columns
Non-categorical attributes cannot be utilized for the discovery of action rules.

In [2]:
stop = ['title', 'abstract']

In [3]:
data = data.drop(columns=stop)

## Create groups of features
The features need to be grouped, and within these groups, we recommend the changes. The package is designed to expect a single value per attribute in each transaction. However, in this case, since multiple values exist, in the action-rules package, a fit_onehot method was developed to handle multiple values for an attribute.

In [4]:
research = ['quantitative_research', 
            'experiment', 
            'exploratory_research', 
            'qualitative_research', 
            'mixed_methods', 
            'observation', 
            'applied_research', 
            'descriptive_research', 
            'focus_groups', 
            'survey_methodology', 
            'correlational_research', 
            'interviews', 
            'questionnaires', 
            'secondary_research', 
            'conclusive_research', 
            'ethnographic_research']
area = ['mathematics',
        #'computer_and_information_sciences',
        'physical_sciences',
        'chemical_sciences',
        #'earth_and_related_environmental_sciences',
        'biological_sciences',
        'other_natural_sciences',
        'civil_engineering',
        'electrical_engineering',
        'mechanical_engineering',
        'chemical_engineering',
        'materials_engineering',
        'environmental_engineering',
        'evironmental_biotechnology',
        'industrial_biotechnology',
        'nano-technology',
        'other_engineering_and_technologies',
        'basic_medicine',
        'clinical_medicine',
        'health_sciences',
        'medical_biotechnology',
        'other_medical_sciences',
        'agriculture_forestry_and_fisheries',
        'animal_and_dairy_science',
        'veterinary_science',
        'agricultural_biotechnology',
        'other_agricultural_sciences',
        'psychology',
        'economics_and_business',
        'education',
        'sociology',
        'law',
        'political_science',
        'social_and_economic_geography',
        'media_and_communications',
        'other_social_sciences',
        'history_and_archaeology',
        'languages_and_literature',
        'philosophy_ethics_and_religion',
        'arts',
        'other_humanities_and_the_arts']

## Onehot encoding
One-hot encoding is a method used to convert categorical variables into a binary vector representation. In this encoding scheme, each category is represented by a vector in which one element is set to 1 (indicating the presence of that category) and all other elements are set to 0. This approach allows categorical data to be used in machine learning models that require numerical input.

In [5]:
labels_to_onehot = ['novelty', 'rigor', 'grammar',	'replicability', 'accessibility', 'cited']
data[labels_to_onehot] = data[labels_to_onehot].astype('string')
data_flexible = pd.get_dummies( 
        data[labels_to_onehot], sparse=False, prefix_sep='_'
    )
target_cols = ['cited']
data_target = pd.get_dummies(data[target_cols], sparse=False, prefix_sep='_')
rest_data = data[research + area]

In [6]:
df = pd.concat([data_flexible, rest_data, data_target], axis=1)
df = df.astype('bool')
df

,novelty_high,novelty_low,novelty_medium,rigor_high,rigor_low,rigor_medium,grammar_0,grammar_1,replicability_0,replicability_1,accessibility_high,accessibility_medium,cited_0,cited_1,quantitative_research,experiment,exploratory_research,qualitative_research,mixed_methods,observation,applied_research,descriptive_research,focus_groups,survey_methodology,correlational_research,interviews,questionnaires,secondary_research,conclusive_research,ethnographic_research,mathematics,physical_sciences,chemical_sciences,biological_sciences,other_natural_sciences,civil_engineering,electrical_engineering,mechanical_engineering,chemical_engineering,materials_engineering,environmental_engineering,evironmental_biotechnology,industrial_biotechnology,nano-technology,other_engineering_and_technologies,basic_medicine,clinical_medicine,health_sciences,medical_biotechnology,other_medical_sciences,agriculture_forestry_and_fisheries,animal_and_dairy_science,veterinary_science,agricultural_biotechnology,other_agricultural_sciences,psychology,economics_and_business,education,sociology,law,political_science,social_and_economic_geography,media_and_communications,other_social_sciences,history_and_archaeology,languages_and_literature,philosophy_ethics_and_religion,arts,other_humanities_and_the_arts,cited_0,cited_1
0,False,False,True,False,False,True,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,True,True,True,True,True,True,True,False,False,True,False,False,True,False
2,False,True,False,False,False,True,True,False,True,False,True,False,True,False,True,False,True,True,True,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False
3,False,True,False,False,True,False,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,True,False,False,True,False,False,True,False,False,True,True,False,True,False,True,False,True,True,True,False,True,True,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,True,False,False,True,False,False,True,False,False,True,True,False,False,True,True,False,True,False,True,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,Fals

In [7]:
novelty = ['novelty_high', 'novelty_low', 'novelty_medium']
rigor = ['rigor_high', 'rigor_low', 'rigor_medium']
grammar = ['grammar_0', 'grammar_1']
replicability = ['replicability_0', 'replicability_1']
accessibility = ['accessibility_high', 'accessibility_low', 'accessibility_medium']

### Prepare stable and flexible attributes
For action rule mining, attributes are divided into two categories: stable attributes, which remain unchanged and serve as fixed conditions in the rules, and flexible attributes, where changes are pursued to achieve the desired outcome. The attribute area is classified as stable, while novelty, rigor, grammar, replicability, accessibility, and research are categorized as flexible attributes.

In [8]:
stable = {'area': area}
flexible = {'novelty': novelty, 'rigor': rigor, 'grammar': grammar, 'replicability': replicability, 'accessibility': accessibility, 'research': research}
target = {'cited': ['cited_0', 'cited_1']}

## Experiment 1: Cited 0 -> 1

### ActionRules Miner Initialization

In [9]:
from action_rules import ActionRules
ar = ActionRules(
    min_stable_attributes=0,
    min_flexible_attributes=1,
    min_undesired_support=20,
    min_undesired_confidence=0.7,
    min_desired_support=20,
    min_desired_confidence=0.7,
    verbose=False)

### Model Training

In [10]:
ar.fit_onehot(
    data=df,           
    stable_attributes=stable,
    flexible_attributes=flexible,
    target=target,
    target_undesired_state='cited_0',
    target_desired_state='cited_1',
    use_sparse_matrix=False,
    use_gpu=True)

### Print

In [11]:
r = ar.get_rules().get_ar_notation()
print('Found: '+str(len(r))+' action rules')
print()
for action_rule in r:
    print(action_rule)
    print()

Found: 16 action rules

[(area: education) ∧ (rigor: rigor_medium → rigor_high) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 47, confidence of undesired part: 0.746031746031746, support of desired part: 20, confidence of desired part: 0.7692307692307693, uplift: 0.010827731308051524

[(area: mathematics) ∧ (novelty: novelty_high → novelty_medium) ∧ (rigor*: rigor_medium) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 22, confidence of undesired part: 0.7586206896551724, support of desired part: 20, confidence of desired part: 0.9090909090909091, uplift: 0.0064588513554490875

[(area: education) ∧ (rigor: rigor_medium → rigor_high) ∧ (grammar*: grammar_0) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 41, confidence of undesired part: 0.7321428571428571, support of desired part: 20, confidence of

### Dominant action rules
Dominant action rules code filters out redundant rules, ensuring that only the most meaningful and relevant rules are retained.

In [12]:
def get_important_rules(json_rules, notation_rules):
    important_rules = []
    
    def create_stable_set(values):
        stable_tuples = [(d["attribute"], d["value"]) for d in values]
        return set(stable_tuples)
    
    def create_flexible_set(values):
        flexible_tuples = [(d["attribute"], d["undesired"], d["desired"]) for d in values]
        return set(flexible_tuples)
    
    candidate_set = create_stable_set(json_rules[0]['stable']) | create_flexible_set(json_rules[0]['flexible'])
    candidate = json_rules[0]
    candidate['to_delete'] = False
    candidate['candidate_set'] = candidate_set
    candidate['rule_index'] = 0
    important_rules.append(candidate)
    
    i = 0
    for candidate in json_rules[1:]:
        i += 1
        candidate_set = create_stable_set(candidate['stable']) | create_flexible_set(candidate['flexible'])
        is_add_rule = True
        for important_rule in important_rules:
            if important_rule['to_delete']:
                continue
            if important_rule['candidate_set'] < candidate_set and important_rule['uplift'] >= candidate['uplift']:
                is_add_rule = False
            elif important_rule['candidate_set'] > candidate_set and important_rule['uplift'] <= candidate['uplift']:
                important_rule['to_delete'] = True
        if is_add_rule:
            candidate['to_delete'] = False
            candidate['candidate_set'] = candidate_set
            candidate['rule_index'] = i
            important_rules.append(candidate)
                
    important_rules = [rule for rule in important_rules if not rule['to_delete']]
    sorted_indices = sorted(important_rules, key=lambda x: x["uplift"], reverse=True)
    important_rules_indices = [rule['rule_index'] for rule in sorted_indices]
    for i in important_rules_indices:
        print(notation_rules[i])
        print()  

In [13]:
rules_list = json.loads(ar.get_rules().get_export_notation())
get_important_rules(rules_list, r)

[(area: education) ∧ (rigor: rigor_medium → rigor_high) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 47, confidence of undesired part: 0.746031746031746, support of desired part: 20, confidence of desired part: 0.7692307692307693, uplift: 0.010827731308051524

[(area: sociology) ∧ (novelty*: novelty_high) ∧ (rigor: rigor_medium → rigor_high) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 44, confidence of undesired part: 0.7333333333333333, support of desired part: 33, confidence of desired part: 0.7333333333333333, uplift: 0.00933955970647098

[(area: media_and_communications) ∧ (novelty*: novelty_high) ∧ (rigor: rigor_medium → rigor_high) ∧ (grammar*: grammar_0) ∧ (replicability: replicability_1 → replicability_0)] ⇒ [cited: cited_0 → cited_1], support of undesired part: 31, confidence of undesired part: 0.7209302325581395, support of desired part: 24, con